In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import json

In [2]:
with open('boxrec_creds.json') as f:
    payload = json.load(f)
session = requests.Session()
LOGIN_URL = 'https://boxrec.com/en/login'
post_response = session.post(LOGIN_URL,data=payload)

In [3]:
class Boxer:
    
    fight_url_base = 'https://boxrec.com/en/proboxer/{id}'
    
    
    def __init__(self, name, id, session, fight_urls=None):
        self.name = name
        self.id = id
        self.session = session
        self.fight_urls = fight_urls if fight_urls else list()
        
    def scrape_urls(self):
        boxer_url = Boxer.fight_url_base.format(id=self.id)
        response = self.session.get(boxer_url)
        assert(response.status_code == 200)
        soup = BeautifulSoup(response.text, 'html.parser')
        for node in soup.find_all('div', class_='boutP'):
            href = node.parent.get('href')
            self.fight_urls.append(href)

In [4]:
fury = Boxer('Tyson Fury', 479205, session)
fury.scrape_urls()
wilder = Boxer('Deontay Wilder', 468841, session)
wilder.scrape_urls()

In [5]:
url = 'https://boxrec.com/en/event/798948/2424982'
response = session.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

In [6]:
table = str(soup.find('table', class_='responseLessDataTable'))
df = pd.read_html(table)[0]
df.head()

,0,1,2
0,referee Kenny Bayless scorecard judges scorecard 53 Glenn Feldman 58 52 Dave Moretti 59 52 Steve Wei...,referee Kenny Bayless scorecard judges scorecard 53 Glenn Feldman 58 52 Dave Moretti 59 52 Steve Wei...,referee Kenny Bayless scorecard judges scorecard 53 Glenn Feldman 58 52 Dave Moretti 59 52 Steve Wei...
1,Deontay Wilder,NaN,Tyson Fury won TKO round 7
2,NaN,ranking,NaN
3,5,NaN,2
4,756.3 points,before fight,549.8 points


In [7]:
def result_from_name(name):
    split = name.split(' won ')
    if len(split) > 1:
        return split[1]

In [8]:
df = (df.iloc[1:16]
      .drop(2)
     )
df.iloc[0, 1] = 'name'
df.iloc[1, 1] = 'ranking'
df = df.set_index(1)
df.loc['result', ] = [result_from_name(x) for x in df.loc['name', ].tolist()]
df.loc['name', ] = [name.split(' won ')[0] for name in df.loc['name', ].tolist()]
df = df.drop('details')
df

,0,2
1,,
name,Deontay Wilder,Tyson Fury
ranking,5,2
before fight,756.3 points,549.8 points
after fight,481.3 points,866.0 points
age,34,31
stance,orthodox,orthodox
height,6′ 7″ / 201cm,6′ 9″ / 206cm
reach,83″ / 211cm,85″ / 216cm
record,NaN,NaN


In [11]:
table = str(soup.find('table', class_='clearTable'))
df_scorecard = pd.read_html(table)[0]
df_scorecard

,0,1,2
0,NaN,referee,NaN
1,NaN,Kenny Bayless,NaN
2,NaN,NaN,NaN
3,scorecard,judges,scorecard
4,53,Glenn Feldman,58
5,52,Dave Moretti,59
6,52,Steve Weisfeld,59
